In [4]:
import sys
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 999
np.set_printoptions(threshold=sys.maxsize)

In [6]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = '/Users/gms/development/nlp/nlpie/projects/acronym/AMIA19_W22_large_scale_nlp/models/w2v_glove_300.txt'
word2vec_output_file = 'w2v.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [8]:
w1 ="stroke"
print(model.most_similar(positive = w1, topn = 6))
#model.most_similar_cosmul(positive=['hepatoma', 'brain'], negative=['liver'])

[('heart', 0.4828336238861084), ('cardiac', 0.45815250277519226), ('debilitating', 0.4523412585258484), ('infarction', 0.4387247562408447), ('illness', 0.4336370527744293), ('complications', 0.43345579504966736)]


In [54]:
file = "/Users/gms/development/nlp/nlpie/projects/acronym/AMIA19_W22_large_scale_nlp/data/stopwords.txt"
with open(file) as f:
    stop_words = f.read().splitlines()

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [55]:
from nltk.tokenize import TreebankWordTokenizer
#from nltk.stem.snowball import SnowballStemmer
def sentence_vector(sentence):
    word_list = TreebankWordTokenizer().tokenize(sentence)
    #stemmer = SnowballStemmer("english", ignore_stopwords=True)
    #word_list = [stemmer.stem(x) for x in word_list]
    word_list = [word for word in word_list if word not in stop_words]
    word_vectors = []
    for x in word_list:
        try:
            word_vectors.append(model[x])
        except KeyError:
            None    
    return sum(word_vectors)/len(word_vectors)
def vector_breakage(sentence):
    word_list = TreebankWordTokenizer().tokenize(sentence)
    #stemmer = SnowballStemmer("english", ignore_stopwords=True)
    #word_list = [stemmer.stem(x) for x in word_list]
    word_list = [word for word in word_list if word not in stop_words]
    word_vectors_list = []
    for x in word_list:
        try:
            if len(model[x])==200:
                word_vectors_list.append(x)
        except:
            None
        else:
            None
    return word_vectors_list

In [56]:
# load full data set
df = pd.read_csv("/Users/gms/development/nlp/nlpie/projects/acronym/AMIA19_W22_large_scale_nlp/data/full.csv")
df = df[['text','expansion']]
df['vec'] = [sentence_vector(x) for x in df.text]
df.expansion.unique()


# load prepartitioned train/test sets
test = pd.read_csv("/Users/gms/development/nlp/nlpie/projects/acronym/AMIA19_W22_large_scale_nlp/data/test.csv")
test = test[['text','expansion']]
test['vec'] = [sentence_vector(x) for x in test.text]

train = pd.read_csv("/Users/gms/development/nlp/nlpie/projects/acronym/AMIA19_W22_large_scale_nlp/data/test.csv")
train = train[['text','expansion']]
train['vec'] = [sentence_vector(x) for x in train.text]

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

In [58]:
X = list(train.vec)
X_train = np.array(X)
y_train = train.expansion

X = list(test.vec)
X_test = np.array(X)
y_test = test.expansion

In [59]:
clf = SVC(C=1.0, kernel='linear', degree=1).fit(X_train, y_train)

In [60]:
pred = clf.predict(X_test)
cm = confusion_matrix(y_test, pred,labels=list(set(df.expansion)))
cross_val_scores = cross_val_score(clf, X, y, cv=7)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


In [61]:
print('accuracy: {}'.format(cross_val_scores))
print()
print(set(df.expansion))
print([len(df[df.expansion == x]) for x in set(df.expansion)])
print()
print(cm)
print()
print(f1_score(y_test,pred,average = 'weighted'))

accuracy: [0.61666667 0.62962963 0.54901961 0.58333333 0.65957447 0.65217391
 0.61363636]

{'morphine sulfate', 'Mall of America:MOA', 'intrathecal', 'mental retardation', 'intertrochanteric', 'immature-to-total neutrophil', 'myocardial infarction:MI', 'UNSURED SENSE', 'United States', '(drug) OR', 'fluorescent in situ hybridization', 'magnetic resonance', 'idiopathic thrombocytopenic purpura:ITP', 'multiples of median', 'mitral stenosis', 'information technology', 'multiple sclerosis', '(drug) MS', 'ischial tuberosity', 'pravastatin or atorvastatin evaluation and infection therapy', 'iliotibial', 'medical student', '(drug) IT', 'milk of magnesia', 'operating room', 'ultrasound', 'master of science', 'GENERAL ENGLISH', 'mitral regurgitation', 'musculoskeletal', 'inspiratory time', 'medical record'}
[279, 1, 58, 3, 14, 1, 1, 11, 396, 1, 449, 314, 1, 439, 1, 102, 207, 1, 48, 1, 40, 1, 2, 57, 466, 94, 3, 329, 176, 2, 1, 1]

[[29  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [62]:
sentence = 'Patient was tested for US, cystic fibrosis and other heritable diseases '
'''
sentence = "Procedure went without complications, and the patient was sent to the \
floor postoperatively after he was extubated in the MM. Thoracic surgery was consulted \
on the day of surgery. Gastrografin upper GI study performed on admission showed no leak. \
However, the patient was admitted under the care of thoracic surgery team, and he was kept \
n.p.o. and followed up on daily basis for any change in vital signs, chest pain for another \
upper GI swallow study which was done 7 days after his symptoms started. "
'''

print(vector_breakage(sentence))
print(clf.predict(sentence_vector(sentence).reshape(1, -1)))

[]
['information technology']


## Try

In [32]:
try_out = ''
print(vector_breakage(try_out))
print(clf.predict(sentence_vector(try_out).reshape(1, -1)))

[]


ZeroDivisionError: division by zero